# Imports

In [ ]:
import numpy as np
import pandas as pd

import torch

from captum.robust import PGD

In [ ]:
import sys

sys.path.append('/mnt/home/rheinrich/taaowpf')

from data.cnn.wpf_dataset_germany_all_experiments import WPF_Germany_DataModule
from models.cnn.resnet import WPF_ResNet
from robustness_evaluation.robustness_scores import MSELossBatch

## Set Hyperparameters for Model & Training

In [ ]:
config = {
    'checkpoint_path_normal_training': '/mnt/home/rheinrich/taaowpf/models/cnn/checkpoints_normal_training/best_resnet_model_normal_training_experiment1.ckpt',
    'checkpoint_path_adversarial_training': '/mnt/home/rheinrich/taaowpf/models/cnn/checkpoints_adversarial_training/best_resnet_model_adversarial_training_experiment1.ckpt',
    'forecast_horizon': 8, # 8 hour ahead wind power forecast 
    'n_past_timesteps': 4, # including current time step
    'resnet_version': 34,
    'forecast_version': 'single', #'all',
    'batch_size': 256,
    'num_workers': 32,
    'learning_rate': 0.001,
    'p_adv_training': 0.,
    'eps_adv_training': 0.15,
    'step_num_adv_training': 100,
    'norm_adv_training': 'Linf',
    'target_attacker': [0.25, 0.40, 0.50, 0.60, 0.65, 0.72, 0.78, 0.82], # increasing
    'step_num_noise_attack': 100,
}

# Initialize DataModule

In [ ]:
windspeed_dir = '/mnt/home/rheinrich/taaowpf/data/cnn/wind_speed_100m_germany_res10x10_012018_062021.csv'
windpower_dir = '/mnt/home/rheinrich/taaowpf/data/cnn/windpower_germany_102018_062021.csv'

In [ ]:
datamodule = WPF_Germany_DataModule(windspeed_dir = windspeed_dir,
                                    windpower_dir = windpower_dir, 
                                    forecast_horizon = config['forecast_horizon'],  
                                    n_past_timesteps = config['n_past_timesteps'],
                                    batch_size = config['batch_size'], 
                                    num_workers = config['num_workers'],
                                    experiment = 1,
                                   )

In [ ]:
datamodule.setup()

## Load examplary sample

In [ ]:
input_windspeed, target = datamodule.test_dataset.__getitem__(75)

In [ ]:
input_windspeed = input_windspeed.unsqueeze(0)

#### Destandardized wind speed of the original input sample

In [ ]:
input_windspeed_destandardized = (input_windspeed * datamodule.std_windspeed) + datamodule.mean_windspeed

In [ ]:
input_windspeed_destandardized = input_windspeed_destandardized.squeeze().detach().numpy()

# Initialize Models

## Load best model checkpoint

#### Normal Training

In [ ]:
model_normal_training = WPF_ResNet.load_from_checkpoint(config['checkpoint_path_normal_training'],
                                                       resnet_version = config['resnet_version'],
                                                       forecast_version = config['forecast_version'],
                                                       forecast_horizon = config['forecast_horizon'],
                                                       n_past_timesteps = config['n_past_timesteps'],
                                                       learning_rate= config['learning_rate'],
                                                       p_adv_training = config['p_adv_training'],
                                                       eps_adv_training = config['eps_adv_training'],
                                                       step_num_adv_training = config['step_num_adv_training'],
                                                       norm_adv_training = config['norm_adv_training']
                                                      )

### Set models to evaluation mode

In [ ]:
model_normal_training.eval()

## Model prediction for original input

#### Normal Training

In [ ]:
with torch.no_grad():
    prediction_normal_training = model_normal_training(input_windspeed)

# Adversarial Robustness Evaluation: Example Attack

#### Set lower bound for perturbations, such that perturbed wind speed is never negative

In [ ]:
lower_bound = (0 - datamodule.mean_windspeed) / datamodule.std_windspeed

#### Target of the attacker

In [ ]:
target_attacker = torch.tensor(config['target_attacker'])

#### DataFrame containing the ground truth target, the attacker's target such as the prediction of the model for the original input

##### Normal Training

In [ ]:
results_normal_training = pd.DataFrame(target.detach().numpy(), columns = ['ground truth'])
results_normal_training["attacker's target"] = target_attacker.numpy()
results_normal_training['original prediction'] = prediction_normal_training.detach().numpy()[0]

## Adversarial Robustness: Targeted PGD Attack (example attack)

### Normal Training

#### Initialize PGD Attack

In [ ]:
pgd_normal_training = PGD(model_normal_training, MSELossBatch(), lower_bound = lower_bound)

#### Generate perturbed input

In [ ]:
input_windspeed_pgd_normal_training = pgd_normal_training.perturb(inputs = input_windspeed,
                                                                  radius = config['eps_adv_training'],
                                                                  step_num = config['step_num_adv_training'],
                                                                  step_size = 2 * config['eps_adv_training'] / config['step_num_adv_training'],
                                                                  target = target_attacker.unsqueeze(0), 
                                                                  targeted = True,
                                                                  norm = config['norm_adv_training']
                                                                 )

#### Model prediction for perturbed input

In [ ]:
with torch.no_grad():
    prediction_pgd_normal_training = model_normal_training(input_windspeed_pgd_normal_training)

#### DataFrame with results (ground truth target, attacker's target, prediction for original input, prediction for perturbed input)

In [ ]:
results_pgd_normal_training = results_normal_training.copy()
results_pgd_normal_training['attacked prediction'] = prediction_pgd_normal_training.detach().numpy()[0]

#### Destandardized wind speed of the perturbed input sample

In [ ]:
input_windspeed_pgd_normal_training_destandardized = (input_windspeed_pgd_normal_training * datamodule.std_windspeed) + datamodule.mean_windspeed

In [ ]:
input_windspeed_pgd_normal_training_destandardized = input_windspeed_pgd_normal_training_destandardized.squeeze().detach().numpy()

# Save all results as CSV file

### Predictions

In [ ]:
results_pgd_normal_training.to_csv('germany_example-attack_predictions_pgd_normal-training_eps15e-2.csv', index = False)

### Inputs

In [ ]:
with open('germany_example-attack_inputs_pgd_normal-training_eps15e-2.npy', 'wb') as f:
    np.save(f, input_windspeed_destandardized, allow_pickle = False)
    np.save(f, input_windspeed_pgd_normal_training_destandardized, allow_pickle = False)